In [10]:
import numpy as np
import pandas as pd
import scipy.linalg # For eigvals and lstsq
import time

# Załaduj dane (jak w Twoim kodzie)
# Pamiętaj, że 'labels' to nazwy kolumn z pliku labels
with open("dataset/breast-cancer.labels", "r") as f:
    labels = [line.strip() for line in f.readlines()]
train_data = pd.io.parsers.read_csv("dataset/breast-cancer-train.dat", names=labels)
validate_data = pd.io.parsers.read_csv("dataset/breast-cancer-validate.dat", names=labels)

# Przekształć kolumnę "Malignant/Benign" na wartości liczbowe (1 dla M, -1 dla B)
# To jest spójne z Twoją implementacją least_square
train_data['Malignant/Benign_numeric'] = np.where(train_data['Malignant/Benign'] == "M", 1, -1)
validate_data['Malignant/Benign_numeric'] = np.where(validate_data['Malignant/Benign'] == "M", 1, -1)


def gradient_descent_for_linear_regression(X_train, y_train, X_validate, y_validate, alpha, n_iterations, threshold=0.0):
    """
    Implementacja Gradient Descent dla Liniowej Regresji (z zastosowaniem do klasyfikacji).

    Args:
        X_train (np.ndarray): Macierz cech treningowych (z dodaną kolumną biasu).
        y_train (np.ndarray): Wektor etykiet treningowych (-1 lub 1).
        X_validate (np.ndarray): Macierz cech walidacyjnych (z dodaną kolumną biasu).
        y_validate (np.ndarray): Wektor etykiet walidacyjnych (-1 lub 1).
        alpha (float): Stała ucząca (learning rate).
        n_iterations (int): Liczba iteracji algorytmu Gradient Descent.
        threshold (float): Próg klasyfikacji (domyślnie 0.0 dla -1/1 etykiet).

    Returns:
        tuple: (theta, costs, tp, tn, fp, fn, accuracy)
               theta (np.ndarray): Ostateczne wagi modelu.
               costs (list): Lista wartości funkcji kosztu w kolejnych iteracjach.
               tp, tn, fp, fn (int): Liczby True Positives, True Negatives, False Positives, False Negatives.
               accuracy (float): Dokładność na zbiorze walidacyjnym.
    """
    n_samples, n_features = X_train.shape
    theta = np.zeros(n_features) # Inicjalizacja wag zerami
    costs = []

    for iteration in range(n_iterations):
        # Obliczenie predykcji (liniowej)
        predictions = X_train @ theta

        # Obliczenie błędu
        error = predictions - y_train.flatten() # Upewnij się, że y_train jest jednowymiarowy


        # Obliczenie gradientu funkcji kosztu (MSE)
        # Gradient MSE = (2/n_samples) * X.T @ (X @ theta - y)
        # Tutaj pomijamy 2/n_samples bo to tylko stała i zostanie wchłonięta przez alpha
        # Albo uwzględniamy ją w gradiencie:
        gradient = (2/n_samples) * X_train.T @ error


        # Aktualizacja wag
        theta = theta - alpha * gradient

        # Obliczenie kosztu (MSE) dla monitorowania
        cost = np.mean(error**2) # MSE
        costs.append(cost)

        # Opcjonalnie: wyświetlanie kosztu co N iteracji
        # if iteration % (n_iterations // 10) == 0:
        #     print(f"Iteration {iteration}, Cost: {cost:.4f}")

    # Ocena modelu na zbiorze walidacyjnym
    predictions_validate = X_validate @ theta

    # Próg klasyfikacji
    classified_predictions = np.where(predictions_validate >= threshold, 1, -1)

    # Funkcja do obliczenia metryk (skopiowana z Twojego kodu)
    def calc_acc(p_vec, b_vec):
        tp = np.sum([1 for p, b in zip(p_vec, b_vec) if p > 0 and b > 0]) # p>0 -> klasyfikujemy na M (1)
        tn = np.sum([1 for p, b in zip(p_vec, b_vec) if p <= 0 and b < 0]) # p<=0 -> klasyfikujemy na B (-1)
        fp = np.sum([1 for p, b in zip(p_vec, b_vec) if p > 0 and b <= 0])
        fn = np.sum([1 for p, b in zip(p_vec, b_vec) if p <= 0 and b > 0])
        total = tp + tn + fp + fn
        accuracy = float((tp + tn) / total) if total > 0 else 0.0
        return int(tp), int(tn), int(fp), int(fn), accuracy

    tp, tn, fp, fn, accuracy = calc_acc(classified_predictions, y_validate.flatten())

    return theta, costs, tp, tn, fp, fn, accuracy


def gradient_descent_runner():
    # Przygotowanie danych treningowych
    # Usuń kolumny "patient ID" i oryginalną "Malignant/Benign"
    A = train_data.drop(["patient ID", "Malignant/Benign"], axis=1).values
    # Dodaj kolumnę jedynek dla biasu (intercept term)
    A = np.c_[np.ones(A.shape[0]), A]

    # Przygotowanie danych walidacyjnych
    A_validate = validate_data.drop(["patient ID", "Malignant/Benign"], axis=1).values
    A_validate = np.c_[np.ones(A_validate.shape[0]), A_validate]

    # Etykiety
    b = train_data['Malignant/Benign_numeric'].values.reshape(-1, 1)
    b_validate = validate_data['Malignant/Benign_numeric'].values.reshape(-1, 1)

    # Obliczenie A.T @ A dla wyznaczenia stałej uczącej
    # Tutaj A to już macierz z dodanym biasem
    AtA = A.T @ A

    # Obliczenie wartości własnych
    eigv = scipy.linalg.eigvals(AtA)
    # Zwróć uwagę, że eigvals zwraca wartości zespolone, musimy wziąć część rzeczywistą
    real_eigv = np.real(eigv)

    # Wyznaczenie stałej uczącej alpha
    # Dla stabilności i zbieżności często używa się 1 / (lambda_max) lub 2 / (lambda_max + lambda_min)
    # Zadanie mówi "najmniejszej i największej wartości własnej macierzy A^T A"
    # Dwie popularne formuły to:
    # 1. alpha = 1 / lambda_max
    # 2. alpha = 2 / (lambda_max + lambda_min) (dla optymalnego zbiegania w najgorszym przypadku)
    # Wybieram 1 / lambda_max, jako prostszą i często działającą, ale 2/(l_max+l_min) jest teoretycznie lepsze dla niektórych f-cji
    # Pamiętaj, że lambda_min może być bliskie 0, co może spowodować dużą alphę
    # Jeśli min_eigv jest bardzo małe, lepiej użyć 1/max_eigv
    lambda_max = np.max(real_eigv)
    lambda_min = np.min(real_eigv)

    # Dobór alpha:
    # Jeśli lambda_min jest bliskie 0, 1/(max+min) może być niestabilne
    # Stosuje się również 1/max. Możesz eksperymentować.
    # Upewnij się, że lambda_max jest dodatnie!
    if lambda_max > 0:
        alpha = 1 / lambda_max
        # Możesz zmniejszyć alpha nieznacznie, np. alpha = 0.99 / lambda_max, aby zwiększyć stabilność
        # alpha = 0.99 * alpha
    else:
        print("Warning: lambda_max is not positive. Defaulting alpha to a small value.")
        alpha = 0.001 # Wartość awaryjna

    print(f"Najmniejsza wartość własna A.T @ A: {lambda_min:.4f}")
    print(f"Największa wartość własna A.T @ A: {lambda_max:.4f}")
    print(f"Wyliczona stała ucząca (alpha): {alpha:.6f}")

    n_iterations = 100000 # Liczba iteracji, możesz ją dostosować

    start_time_gd = time.time()
    theta_gd, costs_gd, tp_gd, tn_gd, fp_gd, fn_gd, acc_gd = gradient_descent_for_linear_regression(
        A, b, A_validate, b_validate, alpha, n_iterations
    )
    end_time_gd = time.time()
    time_gd = end_time_gd - start_time_gd

    print("\n--- Wyniki Gradient Descent ---")
    print(f"Wagi (theta_gd): {theta_gd}")
    print(f"Czas obliczeń (GD): {time_gd:.4f} s")
    print(f"TP: {tp_gd}, TN: {tn_gd}, FP: {fp_gd}, FN: {fn_gd}")
    print(f"Dokładność na zbiorze walidacyjnym (GD): {acc_gd:.4f}")

    return (tp_gd, tn_gd, fp_gd, fn_gd, acc_gd), time_gd

# Teraz możesz uruchomić obie funkcje i porównać wyniki



if __name__ == "__main__":
    print("Rozpoczynam Gradient Descent...")
    gd_results, gd_time = gradient_descent_runner()

    print("\n--- Podsumowanie Porównania ---")
    print(f"Metoda:                 | Dokładność | Czas (s) | Złożoność Teoretyczna")
    print(f"------------------------|------------|----------|-----------------------")
    print(f"Gradient Descent        | {gd_results[4]:.4f}   | {gd_time:.4f}  | O(iterations * m * d)")
    print("\n(gdzie m = liczba_próbek, d = liczba_cech)")

    # Możesz tutaj dodać bardziej szczegółowe porównanie np. wykres kosztu GD
    # import matplotlib.pyplot as plt
    # plt.plot(gd_costs)
    # plt.xlabel("Iteration")
    # plt.ylabel("Cost")
    # plt.title("Gradient Descent Cost over Iterations")
    # plt.show()

Rozpoczynam Gradient Descent...
Najmniejsza wartość własna A.T @ A: 0.0003
Największa wartość własna A.T @ A: 545289637.0781
Wyliczona stała ucząca (alpha): 0.000000

--- Wyniki Gradient Descent ---
Wagi (theta_gd): [-7.42051860e-05 -5.31352492e-04 -9.21711928e-04 -3.15476015e-03
 -1.55449841e-03 -5.90130406e-06  1.74146533e-07  5.46860157e-06
  2.84526799e-06 -1.12310824e-05 -4.54034645e-06 -3.22435765e-06
 -8.87580830e-05  5.88519777e-07  1.08467466e-03 -5.42416358e-07
 -4.77966426e-07 -8.35640185e-07 -3.09457665e-07 -1.50651104e-06
 -2.37043947e-07 -5.15357257e-04 -1.14153215e-03 -2.97505261e-03
  1.95150790e-03 -7.40356043e-06  5.88963155e-06  1.16338156e-05
  3.40693920e-06 -1.42680946e-05 -4.35887013e-06  2.76188046e-04]
Czas obliczeń (GD): 3.4509 s
TP: 49, TN: 193, FP: 7, FN: 11
Dokładność na zbiorze walidacyjnym (GD): 0.9308

--- Podsumowanie Porównania ---
Metoda:                 | Dokładność | Czas (s) | Złożoność Teoretyczna
------------------------|------------|----------|-